Christian Erikson

Math 76 Homework 2

# Health-care assistance via probabilistic graphical modeling

### Objective
In this project, you will create a health-care assistance bot that can suggest diagnoses for a set of symptoms based on a probabilistic graphical model.

### Step 1: Review 
Review the code from the Bayesian networks exercise.

### Step 2: Acquire data
View this [research article](https://www.nature.com/articles/ncomms5212) and download its supplementary data sets 1, 2 and 3. These data sets include the occurrences of diseases, symptoms, and their co-occurrences in the scientific literature. (For the purpose of this exercise, we are going to assume that the frequency of co-occurrences of diseases and symptoms in scientific papers is proportional to the co-occurence frequencies of actual disease cases and symptoms.)

### Step 3: Create a Bayesian network
Using commands from the `pgmpy` library, create a Bayesian network in which the probability of exhibiting a symptom is conditional on the probability of having an associated disease. 

### Step 4: Initialize priors
Use the disease occurrence data to assign prior probabilities for diseases.

### Step 5: Calculate conditional probability tables
Use the co-occurrence data to define CPTs for each connected pair of disease and symptoms. (Hint: You may need to assign some occurrences of symptoms to an "idiopathic disease" to create valid CPTs.)

### Step 6:
Create a minimal interface in which your bot asks a users for a list of observed symptoms and then returns the name of the disease that is the most likely match to the symptoms. (Hint: Review the input/output commands that you have used in last week's homework.)

In [1]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import pandas as pd

# Process data in R
robjects.r('''
    # Load Data
    library(data.table)

    conditional = fread(r"(C:/Users/cmeri/OneDrive - Dartmouth College/Math_76/Assignment2/41467_2014_BFncomms5212_MOESM1045_ESM.txt)")

    # Calculate Probability of disease p(disease | ailment)
    disease = fread(r"(C:/Users/cmeri/OneDrive - Dartmouth College/Math_76/Assignment2/41467_2014_BFncomms5212_MOESM1043_ESM.txt)")
    setnames(disease,'PubMed occurrence', 'DiseaseFreq')
    disease = disease[,DiseaseProb:=DiseaseFreq/sum(.SD[,DiseaseFreq])]

    # Join data
    data = merge(conditional,disease[,c('MeSH Disease Term','DiseaseProb', 'DiseaseFreq')],by='MeSH Disease Term')

    # Prevent Cycle Formation in Bayesian Network from Symptom Disease names overlaping
    data = data[,`MeSH Symptom Term`:=paste('Symptom',`MeSH Symptom Term`,sep=' ')]
    data = data[,Repeats:=.N,by='MeSH Symptom Term']

    # Remane Columns
    setnames(data, 'MeSH Disease Term', 'Disease')
    setnames(data, 'MeSH Symptom Term', 'Symptom')
    setnames(data, 'PubMed occurrence', 'Occurrence')

    # Calculate Conditional Probabilities p(symptom | disease)
    data = data[,Conditional:=(Occurrence/DiseaseFreq)]
''')

# Convert R data.table to pandas DataFrame
data = robjects.r['data']
with (robjects.default_converter + pandas2ri.converter).context():
  df = robjects.conversion.get_conversion().rpy2py(data)

# Limit to first 500 diseases because of computational limitations
df = df[df.Disease.isin(df.Disease.unique()[:40].tolist())]
df

R[write to console]: data.table 1.15.4 using 6 threads (see ?getDTthreads).  
R[write to console]: Latest news: r-datatable.com



,Disease,Symptom,Occurrence,TFIDF score,DiseaseProb,DiseaseFreq,Repeats,Conditional
1,22q11 Deletion Syndrome,Symptom Language Development Disorders,1,2.486567,1.366126e-06,14,351,0.071429
2,22q11 Deletion Syndrome,Symptom Mental Retardation,1,0.905447,1.366126e-06,14,1706,0.071429
3,22q11 Deletion Syndrome,Symptom Olfaction Disorders,1,2.288230,1.366126e-06,14,428,0.071429
4,22q11 Deletion Syndrome,Symptom Respiratory Sounds,1,1.639269,1.366126e-06,14,693,0.071429
5,"46, XX Disorders of Sex Development",Symptom Virilism,1,2.227056,4.879021e-07,5,246,0.200000
...,...,...,...,...,...,...,...,...
1189,Acidosis,Symptom Polyuria,1,2.360901,5.781640e-04,5925,398,0.000169
1190,Acidosis,Symptom Proteinuria,8,9.991822,5.781640e-04,5925,1046,0.001350
1191,Acidosis,Symptom Albuminuria,2,4.130173,5.781640e-04,5925,535,0.000338
1192,Acidosis,Symptom Hemoglobinuria,1,2.883522,5.781640e-04,5925,236,0.000169


In [2]:
from pgmpy.models import BayesianNetwork

model = BayesianNetwork()

model.add_nodes_from(df.Disease.unique())
model.add_nodes_from(df.Symptom.unique())

for i in df.Disease.unique():
    connections = df.loc[df['Disease']==i, 'Symptom']
    for j in connections:
        model.add_edge(i,j)


In [3]:
from pgmpy.factors.discrete import TabularCPD

for disease in df.Disease.unique():
    prob = df.loc[df['Disease']==disease, 'DiseaseProb'][0]
    cpd = TabularCPD(variable=disease, variable_card=2, values=[[1-prob],[prob]])
    model.add_cpds(cpd)

In [4]:
from pgmpy.factors.discrete import TabularCPD

def create_noisy_or_cpds(df, model, leak_prob=0.01):
   
    for symptom in df.Symptom.unique():
        diseases = df.loc[df['Symptom'] == symptom, 'Disease'].tolist()
        prob = df.loc[df['Symptom'] == symptom, 'Conditional'].tolist()
    
        num_diseases = len(diseases)
        num_combinations = 2 ** num_diseases
        
        values = []
        for i in range(num_combinations):
            combination = [(i >> j) & 1 for j in range(num_diseases)]
            p_no_effect = 1.0
            for k, disease_active in enumerate(combination):
                if disease_active:
                    p_no_effect *= (1 - prob[k])
            p_symptom = 1 - p_no_effect
            values.append([1 - p_symptom, p_symptom])
        
        values = list(map(list, zip(*values)))
        
        cpd = TabularCPD(variable=symptom, 
                         variable_card=2,
                         values=values, 
                         evidence=diseases, 
                         evidence_card=[2] * num_diseases)
        
        model.add_cpds(cpd)

create_noisy_or_cpds(df[['Disease', 'Symptom', 'Conditional']], model, leak_prob=0.01)


In [5]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)

In [8]:
def healthbot(symptoms):

    evidence = {symptom: 1 for symptom in symptoms}

    diseases = df['Disease'].unique()
    disease_probs = {}
    for disease in diseases:
        query_result = infer.query(variables=[disease], evidence=evidence)
        disease_probs[disease] = query_result.values[1]

    maxLiklihood = max(disease_probs, key=disease_probs.get)
    
    return maxLiklihood, disease_probs


if __name__ == "__main__":
    
    # Prompt user for symptoms
    symptoms = input("Enter a list of symptoms separated by commas: ").split(',')

    # Find the most likely disease
    maxLiklihood, disease_probs = healthbot(symptoms)
    
    print(f"The most likely disease is: {maxLiklihood}")

Enter a list of symptoms separated by commas:  Symptom Fever,Symptom Cough


The most likely disease is: Abdominal Injuries


Note: The list of diseases was heavily truncated due to memory limits.